In [2]:
from gptlm import GPT2LM
import torch
import argparse
import pandas as pd

In [3]:
def filter_sent(split_sent, pos):
    words_list = split_sent[: pos] + split_sent[pos + 1:]
    return ' '.join(words_list)

In [4]:
def get_PPL(data):
    all_PPL = []
    from tqdm import tqdm
    for i, sent in enumerate(tqdm(data)):
        split_sent = sent.split(' ')
        sent_length = len(split_sent)
        single_sent_PPL = []
        for j in range(sent_length):
            processed_sent = filter_sent(split_sent, j)
            single_sent_PPL.append(LM(processed_sent))
        all_PPL.append(single_sent_PPL)

    assert len(all_PPL) == len(data)
    return all_PPL

In [5]:
def get_processed_sent(flag_li, orig_sent):
    sent = []
    for i, word in enumerate(orig_sent):
        flag = flag_li[i]
        if flag == 1:
            sent.append(word)
    return ' '.join(sent)

In [6]:
def get_processed_poison_data(all_PPL, data,labels, bar):
    processed_data = []
    for i, PPL_li in enumerate(all_PPL):
        orig_sent = data[i]
        orig_split_sent = orig_sent.split(' ')[:-1]
        assert len(orig_split_sent) == len(PPL_li) - 1

        whole_sentence_PPL = PPL_li[-1]
        processed_PPL_li = [ppl - whole_sentence_PPL for ppl in PPL_li][:-1]
        flag_li = []
        for ppl in processed_PPL_li:
            if ppl <= bar:
                flag_li.append(0)
            else:
                flag_li.append(1)

        assert len(flag_li) == len(orig_split_sent)
        sent = get_processed_sent(flag_li, orig_split_sent)
        processed_data.append((sent, labels[i]))
    assert len(all_PPL) == len(processed_data)
    return processed_data

In [7]:
def filter_df(df):
    data = df.values.tolist()
    sentences = [item[0] for item in data]
    labels = [int(item[1]) for item in data]
    processed_data = [(sentences[i], labels[i]) for i in range(len(labels))]
    raw_sentence = [sent[0] for sent in processed_data]
    all_PPL = get_PPL(raw_sentence)
    test_data_filtered = get_processed_poison_data(all_PPL, raw_sentence,labels, -50)
    return test_data_filtered

In [8]:
LM = GPT2LM(use_tf=False, device='cuda' if torch.cuda.is_available() else 'cpu')

print('triglen 1 filter processing')
emotion_test_triglen_1 = filter_df(pd.read_csv('data/emotion_poisoned_test_triglen_1.csv'))
emotion_test_triglen_1 = pd.DataFrame(emotion_test_triglen_1, columns=['text', 'label'])
emotion_test_triglen_1.to_csv('data/emotion_poisoned_test_triglen_1_filtered.csv', index=False)
print('triglen 1 filter done')

print('triglen 2 filter processing')
emotion_test_triglen_2 = filter_df(pd.read_csv('data/emotion_poisoned_test_triglen_2.csv'))
emotion_test_triglen_2 = pd.DataFrame(emotion_test_triglen_2, columns=['text', 'label'])
emotion_test_triglen_2.to_csv('data/emotion_poisoned_test_triglen_2_filtered.csv', index=False)
print('triglen 2 filter done')

print('triglen 3 filter processing')
emotion_test_triglen_3 = filter_df(pd.read_csv('data/emotion_poisoned_test_triglen_3.csv'))
emotion_test_triglen_3 = pd.DataFrame(emotion_test_triglen_3, columns=['text', 'label'])
emotion_test_triglen_3.to_csv('data/emotion_poisoned_test_triglen_3_filtered.csv', index=False)
print('triglen 3 filter done')

/u/grads/amiah/Desktop/codes/transformer architecture attack/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /u/grads/amiah/Desktop/codes/transformer architecture attack/venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/u/grads/amiah/Desktop/codes/transformer architecture attack/venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /u/grads/amiah/Desktop/codes/transformer architecture attack/venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/u/grads/amiah/Desktop/codes/transformer architecture attack/venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/u/grads/amiah/Desktop/codes/transformer architecture attack/venv/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('America/New_York')}
  warn(msg)
/u/grads/amiah/Desktop/codes/transformer architecture attack/venv/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/lib64/sbcl/src')}
  warn(msg)
/u/grads/amiah/Desktop/codes/transformer architecture attack/venv/lib/python

triglen 1 filter processing
triglen 1 filter done
triglen 2 filter processing
triglen 2 filter done
triglen 3 filter processing


100%|███████████████████████████████████████| 4000/4000 [43:34<00:00,  1.53it/s]

triglen 3 filter done
